# A very simple demo to explain how the variogram effects stochastic realizations

In this simple notebook, we exploit the speed of `PyPestUtils` to help build understanding of how the variogram parameters effect the resulting stochastic field

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
sys.path.append("..")
from pypestutils.pestutilslib import PestUtilsLib
lib = PestUtilsLib(logger_level=0)

Just some definitions.  We will use a 100 X 100 grid of nodes with  delx/dely (and an area) of 1.

In [ ]:
nrow = ncol = 100
x = np.cumsum(np.ones((nrow,ncol)),axis=1)
y = np.cumsum(np.ones((nrow,ncol)),axis=0)
area = 1
active = 1
mean = 1 #log10 HK mean
var = 0.1 #log10 HK variance
ec = x.flatten()
nc = y.flatten()
transtype = 0 #not log transformed
vtype = 1 #exponential variogram
power = 0
nreal = 1

This is the function that will generate one realization and plot it for us, given a correlation length (`aa`), an anisotropy ratio (`anis`), and a `bearing`) 

In [ ]:

def plot_real(corrlen,anisotropy,bearing):
    fig, axes = plt.subplots(1,1,figsize=(7,6))
    #axes.clear()
    #reset the random seed so that the underlying pseudo random numbers dont change
    lib.initialize_randgen(1123455)
    # generate one realization
    arrs = lib.fieldgen2d_sva(ec,nc,area,int(active),mean,var,corrlen,anisotropy,bearing,transtype,vtype,power,nreal)
    # plot
    
    cb = axes.imshow(arrs[:,0].reshape((nrow,ncol)),vmin=0.0,vmax=2,cmap="magma")
    #$plt.colorbar(cb,ax=axes,label="$log_{10}$ HK")
    axes.set_xlabel("column")
    axes.set_ylabel("row")
    axes.set_title("a random realization of HK",loc="left")
    plt.show()
                   
        

Move these slowly so it doesnt flicker...

 - corrlen = the correlation length of the variogram
 - anisotropy = the anisotropy ratio of the primary to second axes of the anisotropy ellipse
 - bearing = the angle from north of the primary axis of the anisotropy ellipse

In [ ]:
_ = interact(plot_real,corrlen=(1,50,1),anisotropy=(1,10,0.5),bearing=(0,90,1))